In [1]:
import pandas as pd
import re
import numpy as np
from collections import Counter
from imblearn import over_sampling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  MultiLabelBinarizer,LabelBinarizer
import emoji

In [9]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     |████████████████████████████████| 170 kB 6.4 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=d8299afd6e9a78c7081371b2bd10ad45e0fdacd509edd8d0b415991e94d1f703
  Stored in directory: /home/ec2-user/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [26]:
def emoji_idx(s):
    res=[]
    try:
        co = re.compile(u'['u'\U0001F300-\U0001F64F' u'\U0001F680-\U0001F6FF'u'\u2600-\u2B55]+')
    except re.error:
        co = re.compile(u'('u'\ud83c[\udf00-\udfff]|'u'\ud83d[\udc00-\ude4f\ude80-\udeff]|'u'[\u2600-\u2B55])+')
    it = co.finditer(s)
    for i in it:
        res.append(i.span())
    return res

def correct_idx(s,lab):
    emj=emoji_idx(s)   
    if len(emj)==0 or lab=='[]':
        return lab
    lab=eval(lab)
    # print(lab)
    for i in range(len(lab)):
        lab[i]=list(lab[i])
        lab[i][4]=list(lab[i][4])
        lab[i][5]=list(lab[i][5])
        for j in range(len(emj)):
            # print(1)
            # print(s)
            # print(emj)
            # break
            if emj[j][0]<=lab[i][4][0]:
                lab[i][4][0]-=1
                lab[i][4][1]-=1
            if emj[j][0]<=lab[i][5][0]:
                lab[i][5][0]-=1
                lab[i][5][1]-=1
        lab[i][4]=(lab[i][4][0],lab[i][4][1])
        lab[i][5]=(lab[i][5][0],lab[i][5][1])
        lab[i]=(lab[i][0], lab[i][1],lab[i][2],lab[i][3],lab[i][4],lab[i][5])
    lab=str(lab)
    return lab
def clean(path):
    df=pd.read_csv(path)
    false=[]
    false_idx=[]
    for i in range(len(df)):
        lab=eval(df['aspect_opinion_list'][i])
#     print(lab)
#     print(lab[4][0])
        if df['aspect_opinion_list'][i]=='[]':
            continue
        for j in range(len(lab)):
            if  lab[j][0]!=df['reviews'][i][lab[j][4][0]:lab[j][4][1]] or lab[j][2]!=df['reviews'][i][lab[j][5][0]:lab[j][5][1]]:
                false.append(df['reviews'][i])
                false_idx.append(i)
                break   
    return false_idx                                                   

In [32]:
sentiments=['negative','none','positive']

def pick_cn(content):
    content = str(content)
    REG_CN ="[\u4e00-\u9fa5]";#包含中文英文数字
    for i in content:
        if re.match(REG_CN,i) != 'none':
            return ''.join(re.findall(REG_CN,content))
        else:
            return 'none'
def replace_emoji(content):
    data = content.map(lambda x: emoji.demojize(x))
    res = re.sub('\:.*?\:','xx',data)
    return res

def del_blank(content):
    separators=[' ']
    for separator in separators:
        content=content.replace(separator, '')
    return content

def write_txt(df,path='data/ctrip/train_sample_emoji.txt'):
    with open(path,'a')as f:
        for i in range(df.shape[0]):
            f.write(list(df[i])[0])
            f.write('\n')
def write_txt2(df,false,path='data/ctrip/total1119_clean.txt'):
    with open(path,'a')as f:
        for i in range(len(df)):
            if len(df.loc[i,'reviews'])>600 or i in false:
                continue
            f.write("{} ####{}".format(df.loc[i,'reviews'],df.loc[i,'aspect_opinion_list']))
            f.write('\n')
def read_txt(root):
    res=[]
    with open(root, "r") as f:
        for line in f.readlines():
            line = line.strip('\n')  #去掉列表中每一个元素的换行符
            res.append(line)
    return res
def filter_emoji(desstr,restr=''):
    #过滤表情
    try:
        co = re.compile(u'[\U00010000-\U0010ffff]')
    except re.error:
        co = re.compile(u'[\uD800-\uDBFF][\uDC00-\uDFFF]')
    return co.sub(restr, desstr)
def write_train_test(train_path,test_path,root='data/ctrip/total1109.txt'):
    data=read_txt(root)
    x_train,x_test=train_test_split(data,test_size=0.2, random_state=42)
    with open(train_path,'a')as f1:
        for i in range(len(x_train)):
            f1.write(x_train[i])
            f1.write('\n')
    with open(test_path,'a')as f2:
        for i in range(len(x_test)):
            f2.write(x_test[i])
            f2.write('\n')
            
def del_idx(content):

    return

In [33]:
# df=pd.read_csv('data/ctrip/data1119.csv')
# df['aspect_opinion_list']=df.apply(lambda x:correct_idx(x.reviews,x.aspect_opinion_list),axis=1)
# df.to_csv('data/ctrip/1119clean.csv',index=False)
# false=clean('data/ctrip/1119clean.csv')
write_txt2(df,false)

In [34]:
write_train_test(train_path='data/ctrip/data_total_1119_clean_train.txt',test_path='data/ctrip/data_total_1119_clean_test.txt',root='data/ctrip/total1119_clean.txt')

In [36]:
a=read_txt('data/ctrip/data_total_1119_clean_train.txt')
print(len(a))
a=read_txt('data/ctrip/data_total_1119_clean_test.txt')
print(len(a))
a=read_txt('data/ctrip/total1119_clean.txt')
print(len(a))

17593
4399
21992


In [62]:
labels=np.load('labels.npy',allow_pickle=True)

In [1]:
!nvidia-smi

Mon Nov 29 08:00:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.142.00   Driver Version: 450.142.00   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1E.0 Off |                    0 |
| N/A   73C    P0   256W / 300W |  12283MiB / 16160MiB |     95%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
devtmpfs         30G     0   30G   0% /dev
tmpfs            30G  4.0K   30G   1% /dev/shm
tmpfs            30G  644K   30G   1% /run
tmpfs            30G     0   30G   0% /sys/fs/cgroup
/dev/xvda1      130G  115G   16G  88% /
/dev/xvdf       493G  199G  269G  43% /home/ec2-user/SageMaker
tmpfs           6.0G     0  6.0G   0% /run/user/1002
tmpfs           6.0G     0  6.0G   0% /run/user/1000
tmpfs           6.0G     0  6.0G   0% /run/user/1001


In [2]:
a='客栈到码头非常近。码头下船以后出门左边，转一个湾就到，到普济寺也很方便步行几百米，装修比较有风格，住的二楼房间不错，老板很热情，旁边吃饭也很方便，出门就是好几个饭店，还有小卖部。客栈卫生也不错，房间安静，下次过去还住这里。这次是我一个人去的，定了一个双人标间。房间比较紧凑，但是又很有风格。如果有时间真的可以在这里小住一段时间。喜欢做民宿的朋友，强烈给大家推荐普陀山普陀小院客栈。对了，顺便给大家说一下。线路一，客栈出门，到马路边上，往右走，就是去南海观音。也可以到码头坐大巴车，五块钱到南海观音停车场。然后可以从南海观音出来以后，到紫竹林和不肯去观音寺。紫竹林出来以后也可以选择步行到普济寺，也可以做大巴车，五块钱到普济寺。线路二，往左走就是去普济寺，也可以坐大巴车，到西山景区，下车往前走200米左右，就是普济寺。普济寺出来以后可以去百子堂。然后可以选择步行或者坐大巴车去南海观音，那还观音出来去紫竹林和不肯去观音寺。然后可以在停车场，坐车去法雨寺，法雨寺出来可以选择，爬山去，慧济寺。慧济寺可以坐索道下山到停车场然后坐车去，善财洞和梵音洞。然后选择做车回码头到客栈。因为客栈离码头很近，所以到什么地方坐车都很方便。以上仅供大家参考'
print(a[465:472])


善财洞和梵音洞


In [ ]:
!sh eval.sh

In [4]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
devtmpfs        241G     0  241G   0% /dev
tmpfs           241G  964M  240G   1% /dev/shm
tmpfs           241G  896K  241G   1% /run
tmpfs           241G     0  241G   0% /sys/fs/cgroup
/dev/xvda1      120G  109G   12G  91% /
/dev/xvdf       493G  116G  352G  25% /home/ec2-user/SageMaker
tmpfs            49G     0   49G   0% /run/user/1002
tmpfs            49G     0   49G   0% /run/user/1000
tmpfs            49G     0   49G   0% /run/user/1001


In [9]:
a=(1,2,3,'4')
a=list(a)
a[1]=str(a[1])
a

[1, '2', 3, '4']

In [16]:
import numpy as np
lab=np.load('./labels2.npy',allow_pickle=True)
pre=np.load('./predictions_nofix2.npy',allow_pickle=True)
print(lab[1])

[('', '')]


In [22]:
a='民族特色, 文化主题, 没做出质感, 负, (3, 6), (7, 11)'
a=a.split(', ')
b=[a[0],a[1],a[2],a[3],a[4]+', '+a[5],a[6]+', '+a[7]]


['民族特色', '文化主题', '没做出质感', '负', '(3, 6)', '(7, 11)']

In [9]:
a='12 34 5'
a[3:5]
a=a[:3]+'3344'+a[5:]
a

'12 3344 5'

In [1]:
def get_annotated_aste_targets(sents, labels):
    senttag2word = {'POS': 'positive', 'NEG': 'negative', 'NEU': 'neutral'}

    annotated_targets = []
    num_sents = len(sents)
    for i in range(num_sents):
        tuples = labels[i]
        # tup: ([2], [5], 'NEG')
        for tup in tuples:
            ap, op, sent = tup[0], tup[1], tup[2]
            op = [sents[i][j] for j in op]
            # multiple OT for one AP
            if '[' in sents[i][ap[0]]:
                # print(i)
                if len(ap) == 1:
                    sents[i][ap[0]] = f"{sents[i][ap[0]][:-1]}, {' '.join(op)}]"
                else:
                    sents[i][ap[-1]] = f"{sents[i][ap[-1]][:-1]}, {' '.join(op)}]"
            else:
                annotation = f"{senttag2word[sent]}|{' '.join(op)}"
                if len(ap) == 1:
                    sents[i][ap[0]] = f"[{sents[i][ap[0]]}|{annotation}]"
                else:
                    sents[i][ap[0]] = f"[{sents[i][ap[0]]}"
                    sents[i][ap[-1]] = f"{sents[i][ap[-1]]}|{annotation}]"
        annotated_targets.append(sents[i])
    return annotated_targets

In [3]:
sents, labels=['Screen is awesome , battery life is good .'],[[([0], [2], 'POS'), ([4, 5], [7], 'POS')]]
sents[0]=sents[0].split(' ')
a=get_annotated_aste_targets(sents, labels)
print(a)

[['[Screen|positive|awesome]', 'is', 'awesome', ',', '[battery', 'life|positive|good]', 'is', 'good', '.']]


In [4]:
a='客栈到码头非常近。码头下船以后出门左边,转一个湾就到,到普济寺也很方便步行几百米,装修比较有风格,住的二楼房间不错,老板很热情,旁边吃饭也很方便,出门就是好几个饭店,还有小卖部。客栈卫生也不错,房间安静,下次过去还住这里。这次是我一个人去的,定了一个双人标间。房间比较紧凑,但是又很有风格。如果有时间真的可以在这里小住一段时间。喜欢做民宿的朋友,强烈给大家推荐'
print(len(a))

180


In [36]:
def read_line_examples_from_file(data_path):
    """
    Read data from file, each line is: sent####labels
    Return List[List[word]], List[Tuple]
    """
    sents, labels, total = [], [], []
    with open(data_path, 'r', encoding='UTF-8') as fp:
        words, labels, total = [], [], []
        for line in fp:
            line = line.strip()
            if line != '':
                total.append(line)
                words, tuples = line.split('####')
                sents.append(words.split())
                # sents.append(words)
                labels.append(eval(tuples))
    print(f"Total examples = {len(sents)}")
    return sents, labels, total
def cal_rate(labs):
    num_dict=dict()
    num_dict[' ']=0
    num=0
    for i in range(len(labs)):
        lab=list(set(labs[i]))
        num+=len(lab)
        if len(lab)==0:
            num+=1
            num_dict[' ']+=1
        for j in range(len(lab)):
            if (lab[j][1],lab[j][3]) not in num_dict.keys():
                num_dict[(lab[j][1],lab[j][3])]=1
            else:
                num_dict[(lab[j][1],lab[j][3])]+=1
    for key,val in num_dict.items():
        num_dict[key]=num_dict[key]/num
    return num_dict,num
def build_xy(total,lab):
    x,y=[],[]
    for i in range(len(lab)):
        if len(lab[i])==0:
            x.append(total[i])
            y.append(' ')
        for j in range(len(lab[i])):
            x.append(total[i])
            y.append((lab[i][j][1],lab[i][j][3]))
    return (x,y)

In [37]:
sents,lab,total=read_line_examples_from_file('./data/tasd-cn/ctrip/train.txt')

Total examples = 17757


In [44]:
dic,num=cal_rate(lab)
keys=list(dic.keys())
# x,y=build_xy(total,lab)
x,labs=build_xy(total,lab)
x=np.array(x)
x=x.reshape((-1,1))
print(x.shape)
y=[]
for i in labs:
    y.append(keys.index(i))

(20020, 1)


In [ ]:
print(y)
# mlb = MultiLabelBinarizer()
# y_ = mlb.fit_transform(y)
# label_ = mlb.inverse_transform(y_)
# print(y_[0])


In [46]:
ros = over_sampling.RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(x, y)
# plt.scatter(x=X_resampled[:,0],y=X_resampled[:,1],c=y_resampled)
# sorted(Counter(y_resampled).items())
X_resampled.shape

(613190, 1)

In [54]:
list(X_resampled[1])

"一路上住了很多家酒店和民宿，这家很满意，必须给5分：到达时比较晚了，门口看不出来里面乾坤，但是一进门就感觉到客栈比较宽广了，实木风格，二层结构，房间比较宽敞明亮安静干净整洁，劳累了一天住进来心里感觉很好👍老板很热情，告诉了一个夜观星象的地方😄 ####[('客栈', '面积', '比较宽广', '正', (54, 56), (56, 60)), ('房间', '面积', '比较宽敞', '正', (72, 74), (74, 78))]"

In [56]:
write_txt(X_resampled)

In [57]:
a=read_txt('data/ctrip/train_sample_emoji.txt')
print(len(a))

613190


In [58]:
a[1]

"一路上住了很多家酒店和民宿，这家很满意，必须给5分：到达时比较晚了，门口看不出来里面乾坤，但是一进门就感觉到客栈比较宽广了，实木风格，二层结构，房间比较宽敞明亮安静干净整洁，劳累了一天住进来心里感觉很好👍老板很热情，告诉了一个夜观星象的地方😄 ####[('客栈', '面积', '比较宽广', '正', (54, 56), (56, 60)), ('房间', '面积', '比较宽敞', '正', (72, 74), (74, 78))]"

In [45]:
a=np.load('./labels.npy',allow_pickle=True)
p=np.load('./predictions_nofix.npy',allow_pickle=True)

In [46]:
p[2]

[('孩子体验感受', '正')]

In [47]:
a[2]

[('儿童玩具', '正')]

In [3]:

from boto3.session import Session
import json
# df=pd.read_csv('./data/hp/summary/news_summary_cleaned_small_test.csv')
# print('原文:',df.loc[0,'text'])
# print('真实标签:',df.loc[0,'headlines'])
data={"data": '早餐一般般，勉勉强强填饱肚子，样式可选性不多，可能是疫情的影响吧。不过酒店的服务不错，五个小孩早餐都送了，点👍。由于酒店历史有点长，所以设施感觉一般般，整体还可以，三钻吧'}
session = Session()
    
runtime = session.client("runtime.sagemaker")
response = runtime.invoke_endpoint(
    EndpointName='absa',
    ContentType="application/json",
    Body=json.dumps(data),
)

result = json.loads(response["Body"].read())
print (result)

{'result': '(小孩早餐, 儿童餐饮, 送了点👍, 正)', 'infer_time': '0:00:00.725859'}
